In [60]:
import pandas as pd
import plotly as pt
import folium
import seaborn as sns
import matplotlib.pyplot as plt

In [61]:
confirmed_cases = pd.read_csv('time_series_covid19_confirmed_global.csv')

country_coordinates = {}

for index,row in confirmed_cases.iterrows():
    if row['Country/Region'] in country_coordinates:
        pass
    else:
        country_coordinates[row['Country/Region']] = (row.Lat, row.Long)

In [62]:
confirmed_cases_by_country = confirmed_cases.groupby("Country/Region").sum(numeric_only=True)\
                                            .drop(columns=['Lat', 'Long'])

In [63]:
def date_converter(date_str):
    month_map = {'1': "January", '2': "February", '3': "March", '4': "April", '5': "May", '6': "June", 
                 '7': "July", '8': "August", '9': "September", '10': "October", '11': "November", '12': "December"}

    split_date = date_str.split('/')

    return month_map[split_date[0]] + split_date[-1]

In [64]:
month_list = []
for i in confirmed_cases_by_country.columns:
    month_list.append(date_converter(i))

confirmed_cases_by_country.columns = month_list

In [65]:
monthly_data_df = pd.DataFrame()

for month in pd.unique(confirmed_cases_by_country.columns):
    x = confirmed_cases_by_country[month]
    monthly_data_df[month] = x.max(axis=1)

monthly_data_df

,January20,February20,March20,April20,May20,June20,July20,August20,September20,October20,...,June22,July22,August22,September22,October22,November22,December22,January23,February23,March23
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,5,166,1827,15180,31445,36628,38248,39354,41334,...,182528,185749,193250,199188,203063,205907,207559,208545,209322,209451
Albania,0,0,243,773,1137,2535,5276,9513,13649,20875,...,280851,312097,329352,332221,332966,333343,333806,334167,334391,334457
Algeria,0,1,716,4006,9394,13907,30394,44494,51530,57942,...,266087,267454,270304,270673,270838,271090,271228,271378,271441,271496
Andorra,0,0,376,745,764,855,925,1176,2050,4756,...,43774,45508,46027,46227,46535,47219,47751,47839,47866,47890
Angola,0,0,7,27,86,284,1148,2654,4972,10805,...,101320,102301,102636,103131,103131,104676,105095,105184,105255,105288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank and Gaza,0,0,119,344,448,2428,11837,22729,39899,53520,...,659853,682096,702332,702768,703036,703036,703228,703228,703228,703228
Winter Olympics 2022,0,0,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
Yemen,0,0,0,6,323,1158,1728,1958,2034,2063,...,11824,11877,11926,11935,11939,11945,11945,11945,11945,11945


In [29]:
nigerian_center=[9.0778,8.6775]
globe = folium.Map(location=nigerian_center, zoom_start=3)
display(globe)